In [1]:
from sklearn.datasets import load_iris
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
import scipy as sp
import sklearn
from sklearn.model_selection import train_test_split
%matplotlib inline

/opt/anaconda3/envs/Titanic/lib/python3.7/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/opt/anaconda3/envs/Titanic/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
#データの読み込み
df_train = pd.read_csv('train.csv')
#読み込んだデータの確認
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [16]:
df_train=df_train.replace("male",0).replace("female",1)

In [17]:
df_train.fillna(df_train.Age.median(),inplace=True)

In [18]:
df_train["FamilySize"] = df_train["SibSp"]+df_train["Parch"]+1
df_train2 = df_train.drop(["Name","SibSp","Parch","Ticket","Fare","Cabin","Embarked"],axis=1)

In [19]:
df_train2.head()

,PassengerId,Survived,Pclass,Sex,Age,FamilySize
0,1,0,3,0,22.0,2
1,2,1,1,1,38.0,2
2,3,1,3,1,26.0,1
3,4,1,1,1,35.0,2
4,5,0,3,0,35.0,1


In [20]:
print(df_train2.values)

[[  1.   0.   3.   0.  22.   2.]
 [  2.   1.   1.   1.  38.   2.]
 [  3.   1.   3.   1.  26.   1.]
 ...
 [889.   0.   3.   1.  28.   4.]
 [890.   1.   1.   0.  26.   1.]
 [891.   0.   3.   0.  32.   1.]]


In [21]:
train_data=df_train2.values
X = train_data[:,2:]
Y = train_data[:,1]

In [22]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)

# 学習
forest = forest.fit(X,Y)

In [25]:
df_test = pd.read_csv("test.csv")
df_test = df_test.replace("male",0).replace("female",1)

df_test["Age"].fillna(df_train.Age.median(),inplace=True)
df_test["FamilySize"]=df_test["SibSp"]+df_test["Parch"]+1
df_test2 = df_test.drop(["Name","SibSp","Parch","Ticket","Fare","Cabin","Embarked"],axis=1)

In [26]:
df_test2.head()

,PassengerId,Pclass,Sex,Age,FamilySize
0,892,3,0,34.5,1
1,893,3,1,47.0,2
2,894,2,0,62.0,1
3,895,3,0,27.0,1
4,896,3,1,22.0,3
